## Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import sklearn

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

from pathlib import Path
from IPython.display import Image, display, Video, HTML
from ipywidgets import interact, widgets

from signlens.params import *
from signlens.preprocessing import data, preprocess
from utils import plot_landmarks, model_utils

# reload automatically python functions outside notebook
%load_ext autoreload
%autoreload 2

2024-03-22 12:01:26.120150: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-22 12:01:27.467031: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Fetch data

In [2]:
# variables loaded from .env
print(f"DATA_FRAC : {DATA_FRAC}")
print(f"NUM_CLASSES : {NUM_CLASSES}")
print(f"MAX_SEQ_LEN : {MAX_SEQ_LEN}")

DATA_FRAC : 1.0
NUM_CLASSES : 10
MAX_SEQ_LEN : 100


In [96]:
sequences_data = data.load_data_subset_csv(noface=True, balanced=True, n_classes=NUM_CLASSES)

Loading data subset from train_train.csv
✅ Filtered on n_frames = 100. Size reduced from 94461 to 85660 (90.7%)
✅ Filtered on n_classes = 10. Size reduced from 85660 to 3469 (4.0%)
⚠️ Total size smaller than requested, with 330 per sign instead of 346
✅ Balanced data, with average of 330.0 elements per class. Size reduced from 3469 to 3300 (95.1%)
✅ Loaded 3469 rows (3.5% of the original 94461 rows) from the dataset.


In [97]:
sequences_data.sign.value_counts()

sign
no        330
chair     330
before    330
drink     330
fine      330
go        330
who       330
all       330
book      330
yes       330
Name: count, dtype: int64

In [98]:
from sklearn.model_selection import train_test_split

X_files = sequences_data.file_path
y = preprocess.label_dictionnary(sequences_data)

# Train val split
X_train_files, X_val_files, y_train, y_val = train_test_split(X_files, y, test_size=0.2, stratify=y)


In [ ]:
X_train = preprocess.group_pad_sequences(X_train_files)
X_val = preprocess.group_pad_sequences(X_val_files)


## Model

In [100]:
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import TimeDistributed, LSTM, Dense, Masking, Flatten, Dropout, SimpleRNN, Reshape, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

In [101]:
# 1- RNN Architecture
model = Sequential()

model.add(Reshape((MAX_SEQ_LEN, N_LANDMARKS_NO_FACE * 3),
          input_shape=(MAX_SEQ_LEN, N_LANDMARKS_NO_FACE, 3)))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(units=128, return_sequences=True))
model.add(Dropout(0.5))

model.add(LSTM(units=64))
model.add(Dropout(0.5))

model.add(Dense(NUM_CLASSES, activation='softmax'))  # output layer



model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.summary()

/home/bfrisque/.pyenv/versions/3.10.6/envs/signlens/lib/python3.10/site-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape_12 (Reshape)            │ (None, 100, 225)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ masking_12 (Masking)            │ (None, 100, 225)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_12 (SimpleRNN)       │ (None, 100, 128)       │        45,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_24 (Dropout)            │ (None, 100, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_12 (LSTM)                  │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_25 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 95,370 (372.54 KB)

 Trainable params: 95,370 (372.54 KB)

 Non-trainable params: 0 (0.00 B)

In [102]:
# Fit model

es = EarlyStopping(patience=10, restore_best_weights=True)

history = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=100,
                    batch_size=32, 
                    verbose=1, 
                    callbacks = [es] # This will call the Early Stopping Criterion for each epoch
                   )


Epoch 1/100


2024-03-22 12:58:21.856903: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 237600000 exceeds 10% of free system memory.


83/83 ━━━━━━━━━━━━━━━━━━━━ 22s 228ms/step - accuracy: 0.0859 - loss: 2.3968 - val_accuracy: 0.1045 - val_loss: 2.3016
Epoch 2/100
83/83 ━━━━━━━━━━━━━━━━━━━━ 18s 221ms/step - accuracy: 0.0942 - loss: 2.3194 - val_accuracy: 0.1288 - val_loss: 2.2901
Epoch 3/100
83/83 ━━━━━━━━━━━━━━━━━━━━ 19s 224ms/step - accuracy: 0.1311 - loss: 2.2837 - val_accuracy: 0.1530 - val_loss: 2.2571
Epoch 4/100
83/83 ━━━━━━━━━━━━━━━━━━━━ 16s 192ms/step - accuracy: 0.1393 - loss: 2.2712 - val_accuracy: 0.1742 - val_loss: 2.2263
Epoch 5/100
83/83 ━━━━━━━━━━━━━━━━━━━━ 17s 199ms/step - accuracy: 0.1577 - loss: 2.2400 - val_accuracy: 0.1561 - val_loss: 2.2186
Epoch 6/100
15/83 ━━━━━━━━━━━━━━━━━━━━ 17s 250ms/step - accuracy: 0.1511 - loss: 2.2149


KeyboardInterrupt



In [45]:
model_utils.plot_history_interactive(history)

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='Y Min:', max=1.0), FloatSli…

<function utils.model_utils.plot_history_interactive.<locals>.plot_hist(y_min, y_max, epoch_min, epoch_max)>

## Optimizing imports

In [90]:
def load_and_pad_sequence(file_path, label, n_frames=MAX_SEQ_LEN):
    file_path = file_path.numpy().decode('utf-8')  # Convert tensor to string
    sequence = data.load_relevant_data_subset(file_path)
    sequence = np.pad(sequence, ((0, n_frames - len(sequence)), (0, 0), (0, 0)), mode='constant')
    sequence = sequence[:n_frames]
    return sequence, label

def create_dataset(df, labels, n_frames=MAX_SEQ_LEN, batch_size=32):
    file_paths = df.values
    dataset = tf.data.Dataset.from_tensor_slices((file_paths, labels))
    dataset = dataset.map(tf_load_and_pad_sequence, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.batch(batch_size)  # Batch the dataset
    return dataset

# Create your datasets
train_dataset = create_dataset(X_train_files, y_train)
val_dataset = create_dataset(X_val_files, y_val)

In [91]:
train_dataset

<_BatchDataset element_spec=(TensorSpec(shape=(None, 100, 75, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.float64, name=None))>

In [92]:
# 1- RNN Architecture
model = Sequential()

model.add(Reshape((MAX_SEQ_LEN, N_LANDMARKS_NO_FACE * 3),
          input_shape=(MAX_SEQ_LEN, N_LANDMARKS_NO_FACE, 3)))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(units=128, return_sequences=True))
model.add(Dropout(0.5))

model.add(LSTM(units=64))
model.add(Dropout(0.5))

model.add(Dense(NUM_CLASSES, activation='softmax'))  # output layer



model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.summary()

Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape_10 (Reshape)            │ (None, 100, 225)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ masking_10 (Masking)            │ (None, 100, 225)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_10 (SimpleRNN)       │ (None, 100, 128)       │        45,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_20 (Dropout)            │ (None, 100, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_10 (LSTM)                  │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_21 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 95,370 (372.54 KB)

 Trainable params: 95,370 (372.54 KB)

 Non-trainable params: 0 (0.00 B)

In [93]:

es = EarlyStopping(patience=10, restore_best_weights=True)

history = model.fit(train_dataset,
                    validation_data=val_dataset,
                    epochs=100,
                    batch_size=32, 
                    verbose=1, 
                    callbacks = [es] # This will call the Early Stopping Criterion for each epoch
                   )


Epoch 1/100
22/83 ━━━━━━━━━━━━━━━━━━━━ 52s 865ms/step - accuracy: 0.1123 - loss: 2.4061


KeyboardInterrupt

